Starting with creating a CLI version of my program.

Starting off with creating a simple version of my program that can run with csv files. Does not take into account draft position or how many players your team can fit.

In [37]:
pip install sleeper-py
# found this resource on the internet when looking for sleeper api python on accident, decided to try it out

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# import packages
import pandas as pd
import requests
from pprint import pprint

In [2]:
# constants
api = "https://api.sleeper.app/v1/players/nfl"
user_draft = []

Starting with actual code

In [3]:
# get response from api request
response = requests.get(api)
display(response)
print(response.url)

<Response [200]>

https://api.sleeper.app/v1/players/nfl


In [4]:
player_dict = response.json()
pprint(player_dict['1264'])

with open("players.json", "w+") as fo:
    print(response.text, file = fo)

{'active': True,
 'age': 34,
 'birth_city': None,
 'birth_country': None,
 'birth_date': '1989-11-21',
 'birth_state': None,
 'college': 'Texas',
 'competitions': [],
 'depth_chart_order': 1,
 'depth_chart_position': 'K',
 'espn_id': 15683,
 'fantasy_data_id': 14688,
 'fantasy_positions': ['K'],
 'first_name': 'Justin',
 'full_name': 'Justin Tucker',
 'gsis_id': '00-0029597',
 'hashtag': '#justintucker-NFL-BAL-9',
 'height': '73',
 'high_school': 'Westlake (TX)',
 'injury_body_part': None,
 'injury_notes': None,
 'injury_start_date': None,
 'injury_status': None,
 'last_name': 'Tucker',
 'metadata': {'channel_id': '1113708866371088384', 'rookie_year': '2012'},
 'news_updated': 1727710816642,
 'number': 9,
 'oddsjam_id': '37DEAB30F68A',
 'opta_id': None,
 'pandascore_id': None,
 'player_id': '1264',
 'position': 'K',
 'practice_description': None,
 'practice_participation': None,
 'rotowire_id': 8398,
 'rotoworld_id': 8241,
 'search_first_name': 'justin',
 'search_full_name': 'justintuc

In [5]:
'''
def find_player finds player and keeps track of player

params: player_name
returns: none
'''
def draft(player_name):
    for player in player_dict:
        if player['full_name'] == player_name:
            user_draft.append(player)
            print(f"you have drafted{player_name}")

In [6]:
'''
def calculate_grade calculates draft grade by predicted points

params: list user_draft of players
returns: grade
'''
def calculate_grade(user_draft):
    score = 0
    for player in user_draft:
        score += player['search_rank']

    if score <= 300:
        return "S"
    if score <= 325:
        return "A"
    if score <= 350:
        return "B"
    if score <= 375:
        return "C"
    if score <= 400:
        return "D"
    if score <= 425:
        return "F"

In [8]:
while True:
    player = input("max player size is 5. enter player name here: ")

    draft(player)

    if len(user_draft) > 5:
        break

print("your draft rating is a: " + calculate_grade(user_draft))

TypeError: string indices must be integers, not 'str'